In [1]:
import moviepy.editor as mp
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.signal
import wave, sys
import audio2numpy as a2n
from audio2numpy import open_audio
from scipy.fft import fft, ifft
from scipy.io import wavfile
from scipy.io.wavfile import read
import scipy.io
from pydub import AudioSegment
import pandas as pd
import os
import glob
import ffmpeg

In [2]:
### Define all video paths and file titles

# raw_video_dir = r'\Users\greenhouselab\Documents\Rubi\typingtask_data\pilot_data\p08_marygach_08022022\task_video\right\raw' ## change as needed
# save_dir = r'\Users\greenhouselab\Documents\Rubi\typingtask_data\pilot_data\p08_marygach_08022022\task_video\right\edited' ## change as needed
# whole_video_name = r'p08_08022022_right_wholetask.mp4' ## change to match task subject, date, and camera view
# all_clips_name = r'p08_08022022_right_allclips.mp4'
# audio_name = r'p08_08022022_right.wav' ## change to match task subject, date, and camera view
# video_prefix = 'p08_08022022_right'
# os.chdir(save_dir)

sID = 's262' ## change as needed
cam_view = 'aerial' ## change as needed

server = r'/Volumes/greenhouse/typingtask_data/subject_data/'
os.chdir(server)
sub_folder = glob.glob(f'{sID}_*')[0]
raw_video_dir = server + sub_folder + '/task_video/' + cam_view + '/raw/'
save_dir = server + sub_folder + '/task_video/' + cam_view + '/edited/'
whole_video_name = sub_folder + '_' + cam_view + '_wholetask.mp4'
all_clips_name = sub_folder + '_' + cam_view + '_allclips.mp4'
audio_name = sub_folder + '_' + cam_view + '.wav'
video_prefix = sub_folder + '_' + cam_view
os.chdir(save_dir)

In [3]:
### concatenating all go-pro videos of interest into one
f = open('video_path_file.txt', 'w')
for filename in os.scandir(raw_video_dir):
    if filename.is_file():
        f.write('file \'' + filename.path + '\'')
        f.write('\n')
f.close()

concatenate = 'ffmpeg -f concat -safe 0 -i video_path_file.txt -c copy %s'  % (whole_video_name)
os.system(concatenate) ## output of 0 means it worked :) 

2

In [4]:
### importing whole task video into moviepy format
video_dir = save_dir + whole_video_name
video = mp.VideoFileClip(video_dir)

In [5]:
### extracting audio file from video
audio = video.audio.write_audiofile(audio_name)

AttributeError: type object 'tqdm' has no attribute '_lock'

In [ ]:
### turning stereo audio file to mono
audio = AudioSegment.from_wav(audio_name)
audio = audio.set_channels(1)
audio = audio.export(save_dir + '\\' + audio_name, format="wav")

In [ ]:
### making numpy array from audio file
sample_rate, samples = wavfile.read(audio)

In [ ]:
### Plotting sound file
plt.rcParams["figure.figsize"] = [100, 20]
plt.rcParams["figure.autolayout"] = True
input_data = samples
plt.plot(input_data)
plt.ylabel("Amplitude")
plt.xlabel("Time")
plt.ylim()
plt.xlim()
plt.show()

In [ ]:
# run peak detection to see if there are 240 peaks here
indexes, _ = scipy.signal.find_peaks(samples, height=7, distance=2.1)
print('Peaks are: %s' % (indexes))
print(len(indexes))

In [ ]:
powerSpectrum, frequenciesFound, time, imageAxis = plt.specgram(samples, Fs=sample_rate)
plt.rcParams["figure.figsize"] = [12, 6]
plt.xlabel('Time')
plt.ylabel('Frequency')
plt.ylim(340, 520)
plt.show()

In [ ]:
## need to go through the row that correponds with 440hz (2 and 3 are closest) in powerspectrum, and time stamps of peaks. 
frequenciesFound

In [ ]:
time[2]

In [ ]:
powerSpectrum[3]

In [ ]:
# def find_beep(freq):
#     lowA = pd.()
#     for x in freq:
#         lowA.append(x)

In [ ]:
# find_beep(powerSpectrum[3])

In [ ]:
## other beep is 523.25Hz


In [ ]:
lowA = np.array(powerSpectrum[2])
# plt.rcParams.update({'font.size': 50})
plt.plot(lowA)
plt.xlim()
plt.ylim()
plt.figure(figsize=(120, 20))
plt.show()

In [ ]:
highA = np.array(powerSpectrum[3])
plt.plot(highA)
# plt.figure(figsize=(120, 20))
plt.xlim(200000, 260000)
plt.ylim()
# plt.xticks(np.arange(0, 190000, step=5000))
plt.show()

In [ ]:
indexes, _ = scipy.signal.find_peaks(highA, height=350000, distance=120)
len(indexes)

In [ ]:
indexes

In [ ]:
np.argwhere(indexes == 491663)

In [ ]:
## deleting unwanted peaks (should only be used for data pre-block indicating beep)
indexes = np.delete(indexes, [0, 1, 2])
len(indexes) ## length should always be 240

In [ ]:
## get time stamps for each peak from 'time' output of spectrogram
def find_peak_times(): 
    peak_times = np.empty(np.shape(indexes))
    for idx, x in enumerate(indexes):
        stamp = time[x]
        peak_times[idx] = stamp
    return peak_times

In [ ]:
cut_times = (find_peak_times()).tolist()
len(cut_times) ## should still be 240
cut_times

In [ ]:
for idx, cut in enumerate(cut_times):
    command = 'ffmpeg -t 3.5 -ss %s -i "%s" -y %s_trial_%s.mp4' % (cut, video_dir, video_prefix, idx)
    # print(command)
    os.system(command)

In [ ]:
### concatenating clipped trial videos into one

res = []
for file in os.listdir(save_dir):
    if file.__contains__('trial'):
        res.append(file)
print(len(res))

f = open('clip_path_file.txt', 'w')
for filename in res:
        f.write('file \'' + save_dir + filename + '\'')
        f.write('\n')
f.close()

concatenate = 'ffmpeg -f concat -safe 0 -i clip_path_file.txt -c copy %s'  % (all_clips_name)
os.system(concatenate) ## output of 0 means it worked :) 

In [ ]:
save_dir